In [1]:
from pydantic import BaseModel, ValidationError
from aws_lambda_powertools.utilities.parser import parse
from aws_lambda_powertools.utilities.parser.models import APIGatewayProxyEventModel
from typing import Optional, Dict, Any
import json

# Define the Pydantic model for the 'body' part of the API Gateway event
class PutProduct(BaseModel):
    name: str
    price: float

# Define a custom model extending APIGatewayProxyEventModel
class CreateProductRequest(APIGatewayProxyEventModel):
    body: Optional[PutProduct]  # type: ignore
    pathParameters: Optional[Dict[str, str]]  # type: ignore

# Define a mock API Gateway event with all required fields
mock_event = {
    "resource": "/examplepath",
    "path": "/examplepath",
    "httpMethod": "POST",
    "headers": {},
    "multiValueHeaders": {},
    "queryStringParameters": {},
    "multiValueQueryStringParameters": {},
    "pathParameters": {
        "product": "123e4567-e89b-12d3-a456-426614174000"
    },
    "stageVariables": {},
    "requestContext": {
        "resourceId": "123456",
        "resourcePath": "/examplepath",
        "httpMethod": "POST",
        "extendedRequestId": "1234567",
        "requestTime": "12/Apr/2020:19:34:56 +0000",
        "path": "/prod/examplepath",
        "accountId": "123456789012",
        "protocol": "HTTP/1.1",
        "stage": "prod",
        "domainPrefix": "example",
        "requestTimeEpoch": 1586722096000,
        "requestId": "c6af9ac6-7b61-11e6-9a3e-71f2b348f6d1",
        "identity": {
            "cognitoIdentityPoolId": None,
            "accountId": None,
            "cognitoIdentityId": None,
            "caller": None,
            "sourceIp": "127.0.0.1",
            "principalOrgId": None,
            "accessKey": None,
            "cognitoAuthenticationType": None,
            "cognitoAuthenticationProvider": None,
            "userArn": None,
            "userAgent": "Custom User Agent String",
            "user": None
        },
        "domainName": "example.com",
        "apiId": "1234567890"
    },
    "body": json.dumps({"name": "Widget", "price": 19.99}),
    "isBase64Encoded": False
}

# Lambda handler function
def create_product(event, context) -> Dict[str, Any]:
    try:
        create_input = parse(event=event, model=CreateProductRequest)
        product_id = create_input.pathParameters.get("product", "Unknown")
        product_name = create_input.body.name if create_input.body else "Unknown"
        product_price = create_input.body.price if create_input.body else 0.0

        return {
            "message": "Product created successfully",
            "productId": product_id,
            "productName": product_name,
            "productPrice": product_price
        }
    except ValidationError as e:
        return {"error": str(e)}

# Mock context object (can be expanded as needed)
class LambdaContext:
    function_name = "mock_function"

# Execute the handler function with the mock event and context
result = create_product(mock_event, LambdaContext())
result

{'error': '1 validation error for CreateProductRequest\nbody\n  Input should be a valid dictionary or instance of PutProduct [type=model_type, input_value=\'{"name": "Widget", "price": 19.99}\', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.5/v/model_type'}